**Ho va ten: Do Khanh Dang Khoa**

**MSSV: 19110348**


**Import thư viện**

In [1]:
#import các thư viện
from keras import backend as k
import tensorflow as tf
import numpy as np
#chọn hệ số lamda
regularizer= 0.0005
from keras import applications
from tensorflow import keras


In [2]:
vgg19 = keras.applications.vgg19.VGG19(include_top=True, 
                                       weights=None, 
                                       input_tensor=None, 
                                       input_shape=(32,32,3), 
                                       pooling='max',
                                       classes=10)
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [3]:
## tải CIFAR-100 dataset từ keras
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
##resacle ảnh thành ảnh thực trong đoạn [0,1]
X_train, X_test = X_train/255.0, X_test/255.0

import cv2
## Resize toàn bộ ảnh train tập train và test
X_train = np.array([cv2.resize(img, dsize=(32,32)) for img in X_train])
X_test = np.array([cv2.resize(img, dsize=(32,32)) for img in X_test])
print("Kích thước tập sau khi resize: ", X_train.shape, X_test.shape)

#Tách một phần tập train thành tập valid
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1)

170498071/170498071 [==============================] - 7s 0us/step
Kích thước tập sau khi resize:  (50000, 32, 32, 3) (10000, 32, 32, 3)


**Cấu trúc mô hình**

In [4]:
#Khởi tạo mô hình
model=keras.Sequential()

#input (32,32,3)
model.add(keras.layers.Input(shape=(32,32,3)))

#block1
#Gồm 2 lớp Convolution và 1 lớp maxpooling pool_size là 2x2
#filter 64, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=64,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=64,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding='same',
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                                 strides=[2,2]))

#block2
#Gồm 2 lớp Convolution và 1 lớp maxpooling với pool_size là 2x2
#filter 128, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=128,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=128,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                    strides=[2,2]))

#block3
#Gồm 3 lớp Convolution và 1 lớp maxpooling với pool_size là 2x2
#filter 256, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=256,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=256,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=256,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=256,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                          strides=[2,2]))
#block4
#Gồm 3 lớp Convolution và 1 lớp maxpooling với pool_size là 2x2
#filter 512, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                                 strides=[2,2]))
#block5
#Gồm 4 lớp Convolution và 1 lớp maxpooling với pool_size là 2x2
#filter 512, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                                 strides=[2,2]))

#block 6
#Gồm lớp Flatten để duỗi ra
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=4096,
                             activation=keras.activations.relu,
                             kernel_regularizer=keras.regularizers.l2(regularizer)))
model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Dense(units=4096,
                             activation=keras.activations.relu,
                             kernel_regularizer=keras.regularizers.l2(regularizer)))
model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Dense(units=10,
                             activation=keras.activations.softmax))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),    ##tự khai báo Optimizer với learning rate 10^-4
             loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                    

**Training và Testing**

In [ ]:
### Tạo một callback ModelCheckpoint: callback này sẽ lưu model mỗi khi tìm được một model tốt hơn
#### filepath: đường dẫn file muốn lưu
#### monitor: đại lượng quyết định xem model có "tốt hơn" hay không
#### mode='auto'/'max'/'min': đại lượng monitor lớn hay nhỏ là tốt hơn
#### verbose: có thông báo mỗi lần lưu ko
#### save_best_only: chỉ lưu model tốt nhất
mc = keras.callbacks.ModelCheckpoint(filepath="vgg19_cifar10.h5", 
                                     monitor='val_loss',
                                     mode='min', 
                                     verbose=0, 
                                     save_best_only=True)

## Train  ## Khuyến cáo chạy COLAB (hoặc tương tự)
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs= 25,
                    validation_data=(X_valid, y_valid),
                    callbacks=[mc])                     ##sử dụng callback ModelCheckpoint trong quá trình train


## Đánh giá model trên tập test
valid_loss, valid_acc = model.evaluate(X_valid, y_valid)
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Valid: loss {} acc {} -- Test: loss {} valid {}".format(valid_loss, valid_acc, test_loss, test_acc))

## Load lại model tốt nhất đã lưu
print("best model: ")
model.load_weights("vgg19_cifar10.h5")
valid_loss, valid_acc = model.evaluate(X_valid, y_valid)
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Valid: loss {} acc {} -- Test: loss {} acc {}".format(valid_loss, valid_acc, test_loss, test_acc))

Epoch 1/25
704/704 [==============================] - 51s 58ms/step - loss: 2.5457 - accuracy: 0.1850 - val_loss: 1.9827 - val_accuracy: 0.2442
Epoch 2/25
704/704 [==============================] - 40s 57ms/step - loss: 1.8798 - accuracy: 0.2745 - val_loss: 1.7344 - val_accuracy: 0.3426
Epoch 3/25
704/704 [==============================] - 41s 58ms/step - loss: 1.6093 - accuracy: 0.3901 - val_loss: 1.5484 - val_accuracy: 0.4110
Epoch 4/25
704/704 [==============================] - 41s 58ms/step - loss: 1.3800 - accuracy: 0.5022 - val_loss: 1.2686 - val_accuracy: 0.5484
Epoch 5/25
704/704 [==============================] - 41s 58ms/step - loss: 1.2013 - accuracy: 0.5813 - val_loss: 1.1590 - val_accuracy: 0.5912
Epoch 6/25
704/704 [==============================] - 41s 58ms/step - loss: 1.0454 - accuracy: 0.6386 - val_loss: 0.9995 - val_accuracy: 0.6626
Epoch 7/25
704/704 [==============================] - 41s 58ms/step - loss: 0.9200 - accuracy: 0.6905 - val_loss: 0.9407 - val_accuracy:

In [ ]:
import pandas as pd

d = {'Epoch': [25,50,100,10,50,10], 'Batch size': [64,124,124,100,100,100],
     'Learning rate':[0.0001,0.0001,0.0001,0.0001,0.0001,0.001], 'Weight decay': [0.0005,0.0005,0.0005,0,0,0.0005],
     'Accuracy':[0.77,0.75,0.72,0.73,0.74,0.1]}
df = pd.DataFrame(data=d)
df

,Epoch,Batch size,Learning rate,Weight decay,Accuracy
0,25,64,0.0001,0.0005,0.77
1,50,124,0.0001,0.0005,0.75
2,100,124,0.0001,0.0005,0.72
3,10,100,0.0001,0.0000,0.73
4,50,100,0.0001,0.0000,0.74
5,10,100,0.0010,0.0005,0.10


**Nhận xét**

Dựa vào kết quả ở bảng trên:


*   Khi tăng epoch và batch size lên thì kết quả vẫn cho ra tốt trên 70% nhưng không có sự chênh lệch quá nhiều về mặt hiệu suất
*   Ở trường hợp 2 trường hợp weight decay = 0 thì hiệu suất cho ra trên 70%, ta thấy khi giảm weight decay xuống 0 thì kết quả cho ra vẫn tốt không có sự chênh lệch nhiều
*   Ở trường hợp cuối cùng epoch 10, batch size =100, lrn = 0.001, weight decay = 0.0005. Ta thấy khi giảm lrn xuống 0.001 thì kết quả cho ra chỉ còn 10% khá thấp so với mục tiêu mong muốn.




